# lunasol

In [1]:
from bs4 import BeautifulSoup as bs              # 데이터파싱 라이브러리
from selenium import webdriver
import json
import platform
import re
import copy

C:\Users\dongm\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


# write to json file

In [2]:
def writeJSON(jsonString, output_name='data.json'):
    with open(output_name,'w',encoding='UTF-8') as file:
        file.write(jsonString)


# get item

In [3]:
def getItem(driver): #루나솔 버전
    html = driver.page_source
    soup = bs(html,'lxml')
    
    name = soup.find('div',{'class':'product_summary'}).h1.get_text()    
    url = driver.current_url
    category = soup.find('div',{'id':'directory_link'}).get_text().replace(' > '+name,"")
    image_main = url_home + soup.find('div',{'class':'product_photo'}).img['src']
    
    
    # item 설정
    item = {'name':'', 'url':'', 'image':[], 'color':'', 'category':'', 
                   'salePrice':'', 'originalPrice':'', 'brand':'루나솔'}
    
    item['name']=name
    item['url'] = url
    item['category']=category.strip()
    item['image'].append(image_main)
    
    # return 할 json 설정
    item_json = []
    
    # color로 상품 구분
    colors= soup.find('div',{'class':'product_color'})
    
    if colors: # 색상이 있는 경우
        colors = colors.find_all('img')
        for color in colors:
            image_color = url_home + color['src']
            name_color = color['alt']

            item_dict = copy.deepcopy(item)
            item_dict['image'].append(image_color)
            item_dict['color'] = name_color
            item_json.append(item_dict)
            
    else:
        colors=''
        item_dict = copy.deepcopy(item)
        item_dict['color'] = colors
        item_json.append(item_dict)
        
    print('상품명 :',item_dict['name'])
    print('색상 개수:',len(colors))
    return item_json


# start

In [4]:
path = 'chromedriver.exe' if (platform.system() == 'Windows') else '/Users/jg/Desktop/develop/DataTeam/DataProcessing/product/crawling/chromedriver';
driver = webdriver.Chrome(path)

In [5]:
url_home = 'https://www.lunasol-net.com'
url_product = 'https://www.lunasol-net.com/kr/product/index.html'
driver.get(url_product)

## Open Item

In [6]:
html = driver.page_source
soup = bs(html,'lxml')
category_total = soup.find('div',{'class':'product_mn_list png_fix'})
category_big = category_total.find_all('div')

# crawling start

In [7]:
# open category, item
result = []
for categories in category_big:
    list_category = categories.find_all('li')
    for category in list_category:
        url_category = category.a['href'] # get url of category
        driver.get(url_home + url_category)
        
        html = driver.page_source
        soup = bs(html,'lxml')
        items = soup.find('div',{'class':'product_cat product_cat_first'})
        items = items.find_all('a')
        for item in items:
            driver.get(url_home + item['href'])
            item = getItem(driver)
            result += item
            
        driver.get(url_product)


상품명 : NUANCE SHADE EYES
색상 개수: 4
상품명 : MACARON GLOW EYES
색상 개수: 3
상품명 : SHINE FALL EYES
색상 개수: 3
상품명 : COLORFUL SKY EYES
색상 개수: 2
상품명 : FEATHERY SMOKY EYES
색상 개수: 2
상품명 : SPARKLING EYES
색상 개수: 2
상품명 : SELECTION DE CHOCOLAT EYES
색상 개수: 3
상품명 : GRACE CONTRASTING EYES
색상 개수: 1
상품명 : SAND NATURAL EYES
색상 개수: 2
상품명 : VELVETFUL EYES
색상 개수: 3
상품명 : PETAL PURE EYES
색상 개수: 1
상품명 : THREE-DIMENSIONAL EYES
색상 개수: 2
상품명 : AURORIZED EYES
색상 개수: 1
상품명 : STAR SHOWER EYES
색상 개수: 1
상품명 : SHEER CONTRAST EYES
색상 개수: 3
상품명 : GEMINATE EYES
색상 개수: 2
상품명 : SKIN MODELING EYES
색상 개수: 2
상품명 : SCENT FORM EYES
색상 개수: 1
상품명 : LIGHTING FOR EYES
색상 개수: 1
상품명 : FEATHERY NUANCE EYES
색상 개수: 2
상품명 : SPARKLING LIGHT EYES
색상 개수: 4
상품명 : LIGHTING EYES
색상 개수: 5
상품명 : EYELID BASE(N)
색상 개수: 2
상품명 : FEATHERY LASH MASCARA
색상 개수: 2
상품명 : LASH DEFINING MASCARA WP
색상 개수: 1
상품명 : TREATMENT MASCARA BASE N
색상 개수: 0
상품명 : EYELASH CURLER
색상 개수: 0
상품명 : AIRY GLOW LIPS (NEW)
색상 개수: 4
상품명 : AIRY GLOW LIPS
색상 개수: 5
상품명 : COLORING CRAYON
색상 

In [8]:
output = json.dumps(result,ensure_ascii=False, indent='\t')

writeJSON(output, output_name = 'lunasol.json')